In [21]:
# Imports

# Data curation
import numpy as np
import pandas as pd
import datetime as dt

# Plotting
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.models import LabelSet, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [37]:
# Load the data
df = pd.read_csv('rwc.csv', index_col=0, parse_dates=['Date'])
df

,Date,Type,Distance_km,Hours,Minutes,Seconds,Time_h,Calories,ElevGain_m,AvgSpeed_km/h,Year,Month
0,2015-07-14,Walking,2.10,0,26,40,0.444444,89.0,28.0,4.725000,2015,7
1,2015-07-20,Cycling,21.18,1,8,13,1.136944,332.0,270.0,18.628879,2015,7
2,2015-07-25,Cycling,23.52,1,9,32,1.158889,390.0,327.0,20.295302,2015,7
3,2015-07-27,Running,6.94,0,42,4,0.701111,389.0,97.0,9.898574,2015,7
4,2015-07-29,Walking,1.73,0,20,36,0.343333,69.0,32.0,5.038835,2015,7
...,...,...,...,...,...,...,...,...,...,...,...,...
292,2021-01-05,Running,10.18,0,55,43,0.928611,691.0,213.0,10.962608,2021,1
293,2021-01-08,Cycling,23.50,0,45,0,0.750000,520.0,NaN,31.333333,2021,1
294,2021-01-10,Running,10.18,0,55,9,0.919167,675.0,210.0,11.075249,2021,1
295,2021-01-12,Running,10.15,0,56,3,0.934167,689.0,211.0,10.865299,2021,1


## Add data

In [42]:
def create_new_row(activity=None, year=0, month=0, day=0, distance=0, hours=-1, minutes=-1,
                   seconds=-1, cals=np.nan, elev_gain=np.nan):
    
    """The create_new_row function takes as parameters:
- activity: ['Running', 'Walking', 'Cycling'] - the activity performed by the user;
- year: integer with 4 digits, corresponding to the year when the activity was performed;
- month: integer from 1 to 12, corresponding to the month when the activity was performed;
- day: integer from 1 to 29, 30, or 31, corresponding to the day when the activity was performed;
- distance: positive float corresponding to the number of kms traveled during the activity;
- hours: integer corresponding to the number of hours it took to complete the activity;
- minutes: integer from 0 to 59 corresponding to the number of minutes it took to complete the activity,
not counting the number of hours;
- seconds: integer from 0 to 59 corresponding to the number of seconds it took to complete the activity,
not counting the number of hours and minutes;
- cals: number of calories burned during the activity;
- elev_gain: number of meters climbed during the activity.

    The function will create a new row to be added to the dataframe with all the information from the above
parameters. Then, it will ask if the user wants to add more activities and, when the user does not want to
add more activities, it will ask if the user wants to save the dataframe.

    There are some errors which can arise, especially if the parameters are written beforehand and the process
within the function is not followed. They can be fixed after, but the function works for now if the user leaves
everything as the default values and changes them when the function asks to do so.
    """
    
    # Ensure the activity is either Running, Walking, or Cycling
    while activity not in ['Running', 'Walking', 'Cycling']:
        activity = input('Which activity did you do? [Running, Walking, Cycling] \n')
        
        # If the user writes something different, it will show an error message
        if activity not in ['Running', 'Walking', 'Cycling']:
            print('That is not a valid activity.')
    
    #####################################################################################################
    # Date variables
    #####################################################################################################
    
    # year will accept four digits, corresponding to the year of the activity
    while year == 0:
        
        try:
            year = int(input('Please write the year when you performed the activity: '))
            
            # In case the user writes less than 4 digits, the process restarts
            if len(str(year)) != 4:
                print('Please write only 4 digits.')
                year = 0
        
        # If the user does not write only numbers, it will show an error message
        except ValueError:
            print('Please write only numbers.')
    
    # month will accept  either 1 or 2 digits, corresponding to the month of the activity
    while month == 0:
        
        try:
            month = int(input('Please write the month (number) when you performed the activity: '))
            
            # In case the user writes more than 2 digits or less than 1, the process restarts
            if len(str(month)) not in [1, 2]:
                print('Please write either 1 or 2 digits.')
                month = 0
            
            # If the user writes a number that does not correspond to a month (1-12), the process restarts
            elif month not in list(range(1,13)):
                print('Please write a number from 1 to 12')
                month = 0
        
        # If the user does not write only numbers, it will show an error message
        except ValueError:
            print('Please write only numbers.')
            
    # day will accept  either 1 or 2 digits, corresponding to the day of the activity
    while day == 0:
        
        try:
            day = int(input('Please write the day (number) when you performed the activity: '))
            
            # In case the user writes more than 2 digits or less than 1, the process restarts
            if len(str(day)) not in [1, 2]:
                print('Please write either 1 or 2 digits.')
                day = 0
            
            # In case the selected month has 31 days and the chosen day number is not within the boundaries
            if month in [1, 3, 5, 7, 8, 10, 12] and (day < 0 or day > 31):
                print('Please write a number from 1 to 31')
                day = 0
            
            # In case the selected month has 30 days and the chosen day number is not within the boundaries
            elif month in [4, 6, 9, 11] and (day < 0 or day > 30):
                print('Please write a number from 1 to 30')
                day = 0
            
            # In case the selected month is February and the chosen day number is not within the boundaries
            # Later it should be fixed to accomodate the years with 366 days
            elif month == 2 and (day < 0 or day > 29):
                print('Please write a number from 1 to 30')
                day = 0
        
        # If the user does not write only numbers, it will show an error message
        except ValueError:
            print('Please write only numbers.')
            
    # Create the date
    date = dt.datetime(year, month, day)#.strftime('%Y-%m-%d')
    
    #####################################################################################################
    
    # distance can accept decimal values
    while distance == 0:
        try:
            distance = float(input("""How much was the distance (km) of the activity?
Please separate the decimal part with a ".".\n"""))
            
            # In case the distance written is a negative number, the process restarts
            if distance < 0:
                print('Please write a number larger than 0.')
                distance = 0
    
    # If the user does not write only numbers, it will show an error message
        except ValueError:
            print('Please write only numbers and a decimal point if needed.')
    
    #####################################################################################################
    # Variables linked to the duration of the activity
    #####################################################################################################
    
    # hours will accept an integer corresponding to the number of hours the activity took
    while hours == -1:
        
        try:
            hours = int(input('Please write the amount of hours the activity took: '))
            
            # In case the user writes a negative number, the process restarts
            if hours < 0:
                print('Please write a non-negative number.')
                hours = -1
        
        # If the user does not write only numbers, it will show an error message
        except ValueError:
            print('Please write only numbers.')
            
    # minutes will accept an integer corresponding to the number of minutes the activity took besides the
    # number of hours, i.e., from 0 to 59
    while minutes == -1:
        
        try:
            minutes = int(input('Please write the amount of minutes the activity took (from 0 to 59): '))
            
            # In case the user writes a negative number or a number bigger than 59, the process restarts
            if minutes < 0 or minutes > 59:
                print('Please write a number from 0 to 59.')
                minutes = -1
        
        # If the user does not write only numbers, it will show an error message
        except ValueError:
            print('Please write only numbers.')
            
    # seconds will accept an integer corresponding to the number of seconds the activity took besides the
    # number of hours and minutes, i.e., from 0 to 59
    while seconds == -1:
        
        try:
            seconds = int(input('Please write the amount of seconds the activity took (from 0 to 59): '))
            
            # In case the user writes a negative number or a number bigger than 59, the process restarts
            if seconds < 0 or seconds > 59:
                print('Please write a number from 0 to 59.')
                seconds = -1
        
        # If the user does not write only numbers, it will show an error message
        except ValueError:
            print('Please write only numbers.')
    
    # Create the amount of time spent in the activity in hours
    time = hours+(minutes/60)+(seconds/3600)
    
    # Create the average speed of the activity (km/h)
    avg_speed = distance/time
    
    #####################################################################################################
    # Optional variables
    #####################################################################################################
    
    # Ask if the user wants to add the numbers of calories burned during the activity
    add_calories = input('Do you want to add the number of calories you have burned? [y/n] ')
    
    # If the chosen option is not y or n, ask again until it is one of them
    while add_calories not in ['y', 'n']:
        print('That is not a valid option.')
        add_calories = input('Please write y or n: ')
    
    # If the user wants to add it, the user should write a positive number
    if add_calories == 'y':
        
        while cals == np.nan:
            try:
                cals = int(input('Please write the number of calories you have burned: '))
            
                # if the user writes a negative number, the process restarts
                if cals <= 0:
                    print('Please write a positive number.')
                    cals = np.nan
            
            # If the user does not write only numbers, it will show an error message
            except ValueError:
                print('Please write only numbers.')
    
    # If not, the process continues
    else:
        pass
    
    #####################################################################################################
    
    # Ask if the user wants to add the numbers of meters climbed during the activity
    add_elevation = input('Do you want to add the number of meters you have climbed? [y/n] ')
    
    # If the chosen option is not y or n, ask again until it is one of them
    while add_elevation not in ['y', 'n']:
        print('That is not a valid option.')
        add_elevation = input('Please write y or n: ')
    
    # If the user wants to add it, the user should write a positive number
    if add_elevation == 'y':
        
        while elev_gain == np.nan:
            try:
                elev_gain = int(input('Please write the number of meters you have climbed: '))
            
                # if the user writes a negative number, the process restarts
                if elev_gain <= 0:
                    print('Please write a positive number.')
                    elev_gain = np.nan
            
            # If the user does not write only numbers, it will show an error message
            except ValueError:
                print('Please write only numbers.')
    
    # If not, the process continues
    else:
        pass
    
    #####################################################################################################
    
    # Create a row to add to the dataframe
    last_activity = [date, activity, distance, hours, minutes, seconds, time, cals, elev_gain, avg_speed,
                     year, month]
    
    # Add the row to the dataframe
    df.loc[df.shape[0]] = last_activity
    
    #####################################################################################################
    
    # Ask if there are more activities to be added
    more_activities = input('Do you want to add another activity? [y, n] ')
    
    while more_activities not in ['y', 'n']:
        print('That is not a valid option.')
        more_activities = input('Please write y or n: ')
        
    if more_activities == 'y':
        create_new_row()
    
    # Save the dataframe
    save_data = input('Do you want to save the dataframe? [y, n] ')
    
    while save_data not in ['y', 'n']:
        print('That is not a valid option.')
        save_data = input('Please write y or n: ')
        
    if save_data == 'y':
        df.to_csv('rwc.csv')
    
    return df

## Plots

In [3]:
def yearly_statistics(activity, statistic):
    
    '''
The yearly_statistics function requires 2 arguments: activity, which can be one of the following strings: 
Walking, Running, Cycling; and statistic, which can be one of the following strings: Counter, Distance, Time.

This function produces a bar chart based on the specific activity and statistic in cause, highlighting the
maximum value(s) in red and the minimum value(s) in blue. It also allows for the user to hover the cursor over
the bar to know more information about that year's chosen activity.
    '''
    
    #####################################################################################################
    # Error handling of wrong parameter input
    #####################################################################################################
    
    # Lists to store the options for each parameter
    activity_options = ['Running', 'Walking', 'Cycling']
    statistic_options = ['Distance', 'Time', 'Counter']
    
    # Ensure only one of Running, Walking or Cycling are the accepted activities  
    while activity not in activity_options:
        print('That is not a valid activity.')  # Warning message
        
        # Let the user choose another activity
        activity = input('Please choose one of the following activities [Running, Walking, Cycling]:\n')
        
    # Ensure only one of Distance, Time or Counter can be selected
    while statistic not in statistic_options:
        print('That is not a valid statistic.')  # Warning message
        
        # Let the user choose another statistic
        statistic = input('Please choose one of the following statistics [Distance, Time, Counter]:\n')
    
    #####################################################################################################
    # Data selection and curation
    #####################################################################################################
    
    # Limit the data you will consider based on the activity, group it by year and sum it
    activity_df = df.loc[df.Type==activity].groupby('Year').sum()
    
    # Round the decimal cases of the distance to 2 if the activity is not cycling, and to 0 if it is cycling
    if activity != 'Cycling':
        activity_df.Distance_km = activity_df.Distance_km.round(2)
    else:
        activity_df.Distance_km = activity_df.Distance_km.round(0)
    
    # Add the average speed column which needs to come from the grouped data by years but the mean is taken
    # instead of the sum. In this case, regardless of the activity, the number is rounded to 2 decimal cases
    activity_df['avg_speed'] = df.loc[df.Type==activity].groupby('Year').mean()['AvgSpeed_km/h'].round(2)
    
    # Add the counts column which comes from the grouped data by years and a counter is taken
    activity_df['count'] = df.loc[df.Type==activity].groupby('Year').count()['Date']
    
    # Create a column with the colors of the bars. Green is the smallest, red the biggest and blue are the
    # others. Create also the time labels
    
    color, time_spent = [], [] # Variable to hold the colors and the time labels
    
    for year in activity_df.index: # Loop over the years as they are the indices
        
        # Make sure the colors of the bars are set according to the statistic chosen
        if statistic == 'Distance':
            to_check = activity_df.Distance_km
        elif statistic == 'Time':
            to_check = activity_df.Time_h
        elif statistic == 'Counter':
            to_check = activity_df['count']
    
        # Add the color to the list
        if to_check[year] == min(to_check):
            color.append('green')
        elif to_check[year] == max(to_check):
            color.append('red')
        else:
            color.append('blue')
        
        # Create the time labels
        hour = int(activity_df.Time_h[year]) # The integer part is the number of hours spent
        
        # By removing the integer part to the overall value, you get the minutes, which need to be multiplied
        # by 60 and then rounded to no decimal cases
        minutes = int(round((activity_df.Time_h[year]-hour)*60, 0)) 
        
        time = str(hour)+'h '+str(minutes)+'min' # Create the label
        time_spent.append(time) # Add the label to the list

    # Add the columns to the dataframe
    activity_df['color'] = color
    activity_df['time_spent'] = time_spent
    
    #####################################################################################################
    # Plotting
    #####################################################################################################
    
    # Set the source as the curated dataframe
    source = ColumnDataSource(activity_df)

    # Information when the mouse is hovered over the bars
    tooltips = [('Distance', "@Distance_km{0,0.00} km"), ('Time', "@time_spent"),
                ("Calories burned","@Calories{0,0}"), ("Cumulative Elevation Gain", "@ElevGain_m{0,0} m"),
                ("Average Speed", "@avg_speed{0.00} km/h"), ("Number of runs", "@count")]
    
    # Set the title and the y-axis label
    # If the chosen statistic was Time, the title will only change due to the activity. The label for the 
    # y-axis will always be Hours
    if statistic == 'Time':
        title = 'Amount of Time Spent '+activity # Adapt the title based on the activity
        label = 'Hours' # Y-axis label
    
    # If the chosen statistic is Distance, the title will be adjusted according to the activity, and the
    # y-axis label will be Kilometers
    elif statistic == 'Distance':
        if activity == 'Walking':
            verb = 'Walked'
        elif activity == 'Cycling':
            verb = 'Cycled'
        else:
            verb = 'Run'
        
        # As it happened for Time, the same procedure is applied to Distance
        title = 'Number of Kilometers '+verb+' per Year'
        label = 'Kilometers'
    
    else:
        if activity == 'Walking':
            name = 'Walks'
        elif activity == 'Cycling':
            name = 'Bike Rides'
        else:
            name = 'Runs'
        
        title = 'Number of '+name+' per Year'
        label = 'Number of '+name
    
    # Instantiate the figure
    sports_fig = figure(title=title, x_axis_label='Year', y_axis_label = label, tooltips=tooltips,
                        plot_width=700, plot_height=500, tools='save')

    # Tweak the title
    sports_fig.title.align = 'center'
    sports_fig.title.text_font_size = "20px"

    # Remove unnecessary graph elements
    # Remove gridlines
    sports_fig.xgrid.grid_line_color, sports_fig.ygrid.grid_line_color = None, None

    # Remove x axis minor ticks
    sports_fig.xaxis.minor_tick_line_color = None
    
    # Remove outline line
    sports_fig.outline_line_color = None

    # Vertical bars
    # Set the bar height based on the chosen statistic and choose the data labels accordingly
    if statistic == 'Distance':
        label_choice = height_choice = 'Distance_km'
    elif statistic == 'Time':
        height_choice, label_choice = 'Time_h', 'time_spent'
    else:
        label_choice = height_choice = 'count'
        
    sports_fig.vbar(x='Year', top=height_choice, width=0.9, source=source, color='color')

    # Get the labelslabel_choice = 
    labels = LabelSet(x='Year', y=height_choice, text=label_choice, level='glyph', text_align='center',
                      source=source, render_mode='canvas', y_offset=3)

    # Add the labels to the figure
    sports_fig.add_layout(labels)

    # Show the figure
    show(sports_fig)

In [6]:
yearly_statistics('Walking','Counter')